In [ ]:
import numpy as np
import gym

In [ ]:
env = gym.make('Blackjack-v1', new_step_api=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
Q = np.zeros((32,11,2,2))

In [ ]:
esp = 0.1

In [ ]:
pi = np.full((32,11,2,2),esp/2)

In [ ]:
Returns = np.zeros((32,11,2,2))

In [ ]:
epNum = 1000000

In [ ]:
gamma = 0.9

In [ ]:
from numpy.random import standard_exponential
for episode in range(1,epNum+1):
  state = env.reset()
  done = False
  rewards = []
  stateAction = []
  while True:
    prm1, prm2, prm3 = state
    # print(prm1,prm2,prm3,Q[prm1][prm2][prm3][:])
    if prm3:
      prm3 = 1
    else:
      prm3 = 0
    if np.random.uniform(0,1) > esp:
      action = np.random.choice([0,1])
    else:
      action = np.argmax(Q[prm1][prm2][prm3][:])

    nState, r, terminated, truncated, info = env.step(action)
    rewards.append(r)
    stateAction.append([prm1, prm2, prm3, action])
    state = nState
    if terminated or truncated:
      G = 0
      T = len(rewards)
      # print(rewards)
      for t in range(T-1,-1,-1):
        # print(t)
        G = gamma * G + rewards[t]
        num = Returns[stateAction[t][0]][stateAction[t][1]][stateAction[t][2]][stateAction[t][3]]
        val = Q[stateAction[t][0]][stateAction[t][1]][stateAction[t][2]][stateAction[t][3]]
        avg = (val *num + G)/(num+1)
        Returns[stateAction[t][0]][stateAction[t][1]][stateAction[t][2]][stateAction[t][3]] = num+1
        Q[stateAction[t][0]][stateAction[t][1]][stateAction[t][2]][stateAction[t][3]] = avg
        # print(avg)
        # print(stateAction[t][0],stateAction[t][1],stateAction[t][2],stateAction[t][3])
        # print(Q[stateAction[t][0]][stateAction[t][1]][stateAction[t][2]][stateAction[t][3]])
        best_action = np.argmax(Q[stateAction[t][0]][stateAction[t][1]][stateAction[t][2]][:])
        for a in range(0,2):
          # print("Hi")
          if (a != best_action):
            pi[stateAction[t][0]][stateAction[t][1]][stateAction[t][2]][a] = esp/2
          else:
            pi[stateAction[t][0]][stateAction[t][1]][stateAction[t][2]][a] = 1-esp+esp/2
      break

In [ ]:
win = 0
draw = 0
loss = 0

In [ ]:
for episode in range(1,10000):
  state = env.reset()
  done = False
  rewards = 0
  while True:
    prm1, prm2, prm3 = state
    # print(prm1,prm2,prm3,Q[prm1][prm2][prm3][:])
    if prm3:
      prm3 = 1
    else:
      prm3 = 0

    action = np.argmax(pi[prm1][prm2][prm3][:])

    nState, r, terminated, truncated, info = env.step(action)
    rewards += r
    state = nState
    if terminated or truncated:
      if rewards == 0:
        draw += 1
      elif rewards == 1:
        win+=1
      else:
        loss+=1
      break

In [ ]:
print("Win Percent:", win/(win+draw+loss)*100)
print("Draw Percent:", draw/(win+draw+loss)*100)
print("Loss Percent:", loss/(win+draw+loss)*100)

Win Percent: 43.024302430243026
Draw Percent: 7.01070107010701
Loss Percent: 49.96499649964996
